In [1]:
! apt-get install -y python3-dev
# ! pip install transformers
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 66.3 MB/s 
     |████████████████████████████████| 880 kB 73.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=27caa2499bd53d7338d7b36304be0c970821600b9903191cc07622af4190ad3b
  Stored in directo

In [2]:
import json
import random

import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW

# 1. 데이터 처리

In [6]:
from google.colab import files
uploaded = files.upload()

Saving train.json to train.json


In [7]:
with open("./train.json", 'rb') as f:
    klue_dict = json.load(f)

In [8]:
klue_dict["data"][1]

{'title': '부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정',
 'paragraphs': [{'context': '부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 4개 과제가 선정되어 본격적인 사업 착수에 나선다. 과학기술정보통신부가 주관하는 ‘지역SW서비스사업화 지원사업’은 강소SW기업 및 초기 스타트업의 SW서비스 사업화 지원과 신시장 진출 지원을 통해 기업 경쟁력 강화와 지역경제 활성화를 도모하는 사업이다. 올해부터 2개년으로 진행되며, 국비와 시비, 민자 등 2년간 약 37억원의 예산이 투입된다. 앞서 진흥원은 부산의 미래 먹거리산업인 스마트해양, 지능형기계, 지능정보서비스 분야로 사전 수요조사를 진행했고, 평가를 통해 선정된 5개 과제를 공모사업에 신청했다. 그 결과 부산의 4개 과제가 최종 선정되는 쾌거를 거뒀다. 당 사업은 전국 진흥기관을 대상으로 공모를 시작해, 총 17개 지역에서 42개 과제가 선정되었으며, 4개 과제가 선정된 곳은 부산과 강원지역 뿐이다. 금번 선정된 과제들은 ‘인공지능융합센서와 서보 이송 로봇을 이용한 전단보강재의 자동용접시스템 개발’ 등 총 4개 과제다. 부산시가 지원하고, 부산정보산업진흥원과 지역기업, 대학, 연구소 등이 컨소시엄을 구성하여 기술개발 및 사업화 지원을 추진한다. 2개의 Track으로 구분되는 이번사업은 Track 1(SW중소기업)에서 ㈜에이아이플랫폼, 엔컴(주), Track 2(스타트업)에서는 ㈜토즈, 삼보테크놀로지를 지원한다. ○ ‘Track 1‘의 (주)에이아이플랫폼이 주관기업으로 진행하는 <인공지능 기반 망막 내 아밀로이드 플라크 영상 분석을 통한 치매조기진단 플랫폼 상용화>는 치매 확진의 원인이 되는 중요 단백질(아밀로이드 플라크)을 자체개발 관측장비로 진단한다. 이를 통해 치매를 조기 발견하여, 각종 경제적 비용과 치료 및 예방 등 사회적 문제를 해 결하고 시민들이 쉽게 접근 가능한 실효성 있는 치매관리

In [30]:
# 해당 데이터는 각 context에 대한 여러 question과 answer이 있는 형태이므로 이를 context, question, answer로 분리
import random
def read_klue(path):
    with open(path, 'rb') as f:
        klue_dict = json.load(f)
        data_length = len(klue_dict['data'])
        random.shuffle(klue_dict['data'])
        print(data_length)

    contexts = []
    questions = []
    answers = []

    # train
    for group in tqdm(klue_dict['data']):
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    # return contexts, questions, answers

    train = (contexts[:int(data_length*0.8)], questions[:int(data_length*0.8)], answers[:int(data_length*0.8)])
    dev = (contexts[int(data_length*0.8):], questions[int(data_length*0.8):], answers[int(data_length*0.8):])

    return train, dev

In [10]:
# 현재 정답이 context에서 시작하는 위치(answer_start)만 있기 때문에 끝나는 위치도 구
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2

In [26]:
# Tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
# tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [27]:
question = "북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?"
context = "올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다."
tokenizer(context, question)

{'input_ids': [2, 1446, 22555, 11477, 2116, 3932, 2210, 6530, 27135, 3670, 2367, 2062, 18, 3671, 886, 9775, 16311, 2073, 12982, 2178, 2062, 15513, 3309, 3681, 798, 2073, 5277, 1041, 2678, 11477, 2116, 3670, 2651, 4016, 28674, 18, 3932, 2210, 11945, 2170, 3881, 2460, 6530, 7831, 1060, 10526, 2170, 1513, 2259, 11477, 2165, 2020, 2079, 3979, 6233, 3814, 6530, 24028, 1116, 12468, 17552, 2170, 24902, 3802, 2178, 2116, 23772, 31369, 5844, 2170, 3911, 3569, 2170, 10760, 2205, 2259, 1039, 2073, 1187, 2116, 5740, 2062, 18, 4364, 2079, 11477, 2259, 18673, 2178, 2062, 22, 97, 23, 2210, 16, 3736, 2178, 4000, 4051, 5947, 3670, 2367, 2062, 18, 11477, 2259, 19880, 2062, 2219, 2470, 1174, 18956, 26797, 2145, 1891, 2398, 1322, 2399, 2470, 22152, 2128, 2292, 2097, 26797, 2052, 4026, 4605, 2496, 2259, 11477, 2165, 2020, 27135, 4848, 2259, 1187, 2138, 936, 4538, 18, 11477, 2165, 2020, 2073, 3801, 2210, 6530, 1060, 7831, 7755, 6233, 12314, 4795, 3619, 2210, 2678, 3690, 25848, 2097, 4997, 18787, 2299, 2118,

In [28]:
class KlueDataset(Dataset):
    def __init__(self, contexts, questions, answers, model_max_position_embedings, tokenizer):
        self.tokenizer = tokenizer
        self.answers = answers
        self.questions = questions
        self.contexts = contexts
        self.model_max_position_embedings = model_max_position_embedings
        print("Tokenizing ...")
        self.encodings = self.tokenizer(self.contexts,
                                        self.questions,
                                        max_length=512,
                                        truncation=True,
                                        padding="max_length",
                                        return_token_type_ids=False)
        print("Done !!!")
        self.add_token_positions()

    def add_token_positions(self):
        start_positions = []
        end_positions = []
        for i in range(len(self.answers)):
            start_positions.append(self.encodings.char_to_token(i, self.answers[i]['answer_start']))
            end_positions.append(self.encodings.char_to_token(i, self.answers[i]['answer_end'] - 1))

            # positions 값이 None 값이라면, answer가 포함된 context가 잘렸다는 의미
            if start_positions[-1] is None:
                start_positions[-1] = self.model_max_position_embedings
            if end_positions[-1] is None:
                end_positions[-1] = self.model_max_position_embedings

        self.encodings.update({'start_positions': start_positions, 'end_positions': end_positions})


    def get_data(self):
        return {"contexts":self.contexts, 'questions':self.questions, 'answers':self.answers}


    def get_encodings(self):
        return self.encodings


    def __getitem__(self, idx):
        return {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

In [31]:
# contexts, questions, answers = read_klue("./train.json")
train, dev = read_klue("./train.json")
contexts, questions, answers = train
dev_contexts, dev_questions, dev_answers = dev
add_end_idx(answers, contexts)
add_end_idx(dev_answers, dev_contexts)
train_dataset = KlueDataset(contexts, questions, answers, 512, tokenizer)
dev_dataset = KlueDataset(dev_contexts, dev_questions, dev_answers, 512, tokenizer)

9789


  0%|          | 0/9789 [00:00<?, ?it/s]

Tokenizing ...
Done !!!
Tokenizing ...
Done !!!


# 2. 모델 학습

In [33]:
# model 불러오기
# model = AutoModelForQuestionAnswering.from_pretrained("monologg/kobert")
model = AutoModelForQuestionAnswering.from_pretrained("klue/bert-base")
# model = AutoModelForQuestionAnswering.from_pretrained("bert-base-cased")
# model = AutoModelForQuestionAnswering.from_pretrained("skt/kobert-base-v1")


Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model chec

In [50]:
# 파라미터 정의
EPOCH = 20
LEARNING_RATE = 5e-5
BATCH_SIZE = 32

In [51]:
import os
def my_seed_everywhere(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # numpy
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    # pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed) 
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False 

my_seed = 42
my_seed_everywhere(my_seed)

In [52]:
def train_runner(model, dataset, batch_size, num_train_epochs, learning_rate):
    device = torch.device('cuda')
    model.to(device)
    model.train()
    train_dataloader = DataLoader(dataset=dataset, batch_size=batch_size)
    global_total_step = len(train_dataloader) * num_train_epochs
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0)
    print("TRAIN START")
    with tqdm(total=global_total_step, unit='step') as t:
        total = 0
        total_loss = 0
        for epoch in range(num_train_epochs):
            for batch in train_dataloader:
                optimizer.zero_grad()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                start_positions = batch['start_positions'].to(device)
                end_positions = batch['end_positions'].to(device)
                outputs = model(input_ids,
                             attention_mask=attention_mask,
                             start_positions=start_positions,
                             end_positions=end_positions)
                loss = outputs.loss
                loss.backward()
                optimizer.step()

                batch_loss = loss.item() * len(input_ids)
                total += len(input_ids)
                total_loss += batch_loss
                global_total_step += 1
                t.set_postfix(loss="{:.6f}".format(total_loss / total), batch_loss="{:.6f}".format(batch_loss))
                t.update(1)

                del input_ids
                del attention_mask
                del start_positions
                del end_positions
                del outputs
                del loss
    model.save_pretrained("./klue_output_model")
    print("TRAIN END")

In [53]:
train_runner(model,train_dataset, BATCH_SIZE, EPOCH, LEARNING_RATE)

TRAIN START


  0%|          | 0/4900 [00:00<?, ?step/s]

TRAIN END


In [49]:
# out of memory solution

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

# 3. 모델 평가

In [54]:
def read_dev_klue(path):
    with open(path, 'rb') as f:
        klue_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in tqdm(klue_dict['data']):
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                temp_answer = []
                for answer in qa['answers']:
                    temp_answer.append(answer['text'])
                if len(temp_answer) != 0: # answers의 길이가 0 == 답변할 수 없는 질문
                    contexts.append(context)
                    questions.append(question)
                    answers.append(temp_answer)

    return contexts, questions, answers

In [ ]:
# dev_contexts, dev_questions, dev_answers = read_dev_klue("./klue_dataset/klue-mrc-v1_dev.json")

In [55]:
def prediction(contexts, questions):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    model.to(device)
    model.eval()

    result = []

    with torch.no_grad():
        for context, question in zip(contexts, questions):
            encodings = tokenizer(context, question, max_length=512, truncation=True,
                                     padding="max_length", return_token_type_ids=False)
            encodings = {key: torch.tensor([val]) for key, val in encodings.items()}

            input_ids = encodings["input_ids"].to(device)
            attention_mask = encodings["attention_mask"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            start_logits, end_logits = outputs.start_logits, outputs.end_logits
            token_start_index, token_end_index = start_logits.argmax(dim=-1), end_logits.argmax(dim=-1)
            pred_ids = input_ids[0][token_start_index: token_end_index + 1]
            pred = tokenizer.decode(pred_ids)
            result.append(pred)

    return result

In [56]:
pred_answers = prediction(dev_contexts, dev_questions)
pred_answers

['변희재',
 '변희재',
 '김준수',
 '정의화',
 '1월31일',
 '뉴질랜드',
 '창업기업',
 '트리톤',
 '13개',
 '13개',
 '프로테우스이다. 바깥쪽 둘을 제외한 나머지는 해왕성 동기 궤도 안쪽에 있기 때문에 ( 해왕성의 자전 주기는 0. 6713 일이다. ) 조석적으로 감속되고 있다. 해왕성에서 가장 가까운 규칙 위성 나이아드는 내위성들 중 두 번째로 작다. ( 가장 작은 위성은 히포캠프',
 '1408억6000만달러',
 '1408억6000만달러',
 '10억달러를 조달할 계획이라고 밝혔다. 하지만 이는 서류 제출을 위한 형식상의 수치일 뿐이다. IPO 목표액은 보통 투자설명회 ( 로드쇼 ) 에서 투자자들의 반응을 보고 결정한다. 전문가들은 알리바바가 IPO를 통해 최대 200억달러를 조달할 것으로 보고 있다. 정보기술 ( IT ) 업계 최대 규모였던 페이스북의 2012년 IPO 조달금액 164억달러는 물론 미 증시 사상 최대였던 2008년 비자카드 IPO 조달금액 179억달러도 넘어설 것이란 전망이다. 알리바바 IPO에 뜨거운 관심이 쏠리는 것은 무엇보다 타의 추종을 불허하는 기업 규모 때문이다. 지난해 알리바바 주요 사이트인 타오바오, 티몰 등을 이용해 제품이나 서비스를 구매한 사람은 2억3100만명이다. 계약 규모는 총 2480억달러에 달했다. 핀란드 국내총생산 ( GDP ) 인 2500억달러와 맞먹는 수준이다. 미국 아마존을 통한 계약 규모 1100억달러와 비교하면 두 배 이상 차이가 난다. 게다가 지난해 4분기 알리바바를 통한 전자상거래 규모는 전년 동기보다 53 % 급증했다. 특히 모바일 거래 비중이 1년 전 7. 4 %',
 '10억달러를 조달할 계획이라고 밝혔다. 하지만 이는 서류 제출을 위한 형식상의 수치일 뿐이다. IPO 목표액은 보통 투자설명회 ( 로드쇼 ) 에서 투자자들의 반응을 보고 결정한다. 전문가들은 알리바바가 IPO를 통해 최대 200억달러를 조달할 것으로 보고 있다. 정보기술 ( IT ) 업계 최대 

In [57]:
def em_evalutate(prediction_answers, real_answers):
    total = len(prediction_answers)
    exact_match = 0
    for prediction_answer, real_answer in zip(prediction_answers, real_answers):
        if prediction_answer in real_answer:
            exact_match += 1

    return (exact_match/total) * 100

In [58]:
em_score = em_evalutate(pred_answers, dev_answers)
em_score

0.0

In [45]:
dev_answers

[{'text': '변희재 주간미디어워치 대표', 'answer_start': 197, 'answer_end': 211},
 {'text': '변 대표', 'answer_start': 358, 'answer_end': 362},
 {'text': '숙영', 'answer_start': 422, 'answer_end': 424},
 {'text': '정의화', 'answer_start': 858, 'answer_end': 861},
 {'text': '1월31일', 'answer_start': 240, 'answer_end': 245},
 {'text': '뉴질랜드', 'answer_start': 645, 'answer_end': 649},
 {'text': '창업기업', 'answer_start': 441, 'answer_end': 445},
 {'text': '트리톤', 'answer_start': 62, 'answer_end': 65},
 {'text': '13개', 'answer_start': 526, 'answer_end': 529},
 {'text': '13', 'answer_start': 526, 'answer_end': 528},
 {'text': '내위성', 'answer_start': 251, 'answer_end': 254},
 {'text': '1408억6000만달러', 'answer_start': 177, 'answer_end': 189},
 {'text': '1408억6000만', 'answer_start': 177, 'answer_end': 187},
 {'text': '8.9%', 'answer_start': 1311, 'answer_end': 1315},
 {'text': '8.9', 'answer_start': 1311, 'answer_end': 1314},
 {'text': '그랜드인터컨티넨탈호텔', 'answer_start': 386, 'answer_end': 397},
 {'text': '‘더플라자 콘서트’', 'answer